In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_text, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from copy import deepcopy
from sklearn.utils import class_weight
import joblib
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

2024-10-15 09:34:09.611470: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
    smart_attack_path = 'attack_10_10.csv'

In [3]:
def hex_to_bin(hex_num):
    
    binary_value = bin(int(str(hex_num), 16))[2:]
    
    return binary_value

def int_to_bin(int_num):
    
    binary_value = bin(int_num)[2:]
    
    return binary_value

def pad(value, length):
    
    curr_length = len(str(value))
    
    zeros = '0' * (length - curr_length)
    
    return zeros + value

hex_to_dec = lambda x: int(x, 16)

def transform_data(data):

    data['ID'] = data['ID'].apply(hex_to_dec)
    data['Payload'] = data['Payload'].apply(hex_to_dec)

    return data

In [4]:
def shift_columns(df):
    
    for dlc in [2,5,6]:

        df.loc[df['dlc'] == dlc, df.columns[3:]] = df.loc[df['dlc'] == dlc, df.columns[3:]].shift(periods=8-dlc, axis='columns', fill_value='00')

    return df

In [5]:
def read_attack_data(data_path):
    
    columns = ['timestamp','can_id', 'dlc', 'data0', 'data1', 'data2', 'data3', 'data4', 
           'data5', 'data6', 'data7', 'flag']
    
    data = pd.read_csv(data_path, names = columns)

    data = shift_columns(data)
    
    ##Replacing all NaNs with '00' 
    data = data.replace(np.NaN, '00')
    
    ##Joining all data columns to put all data in one column
    data_cols = ['data0', 'data1', 'data2', 'data3', 'data4', 'data5', 'data6', 'data7']
    
    ##The data column is in hexadecimal
    data['data'] = data[data_cols].apply(''.join, axis=1)
    data.drop(columns = data_cols, inplace = True, axis = 1)
    
    ##Converting columns to decimal
    data['can_id'] = data['can_id'].apply(hex_to_dec)
    data['data'] = data['data'].apply(hex_to_dec)

    data = data.assign(IAT=data['timestamp'].diff().fillna(0))
    
    return data[:150_000]

    

In [6]:
smart_attack = pd.read_csv(smart_attack_path)

In [7]:
def sequencify_data(X, y, seq_size=10):
    max_index = len(X) - seq_size + 1

    X_seq = []
    y_seq = []

    for i in range(0, max_index, seq_size):
        X_seq.append(X[i:i+seq_size])  # Append the sequence from DataFrame 'X'
        try:
            y_seq.append(1 if 1 in y[i:i+seq_size].values else 0)  # Check for '1' in 'y' values
        except:
             y_seq.append(1 if 1 in y[i:i+seq_size] else 0)

    return np.array(X_seq), np.array(y_seq)

In [8]:
smart_attack.drop(columns = ['Timestamp'], inplace = True)

In [9]:
X_smart = smart_attack.drop(['label'], axis = 1).values
y_smart = smart_attack['label']
y_tri = y_smart.copy(deep = True)
y_smart = y_smart.replace(2,1)

In [10]:
np.unique(y_smart, return_counts = True)

(array([0., 1.]), array([100000,  47462]))

In [11]:
X_seq_smart, y_seq_smart = sequencify_data(X_smart, y_smart)

In [12]:
print(np.unique(y_seq_smart, return_counts=True))

(array([0, 1]), array([ 1350, 13396]))


In [13]:
def balance_data(X_seq, y_seq):
    # Get indices for label 0 and label 1
    zero_indices = np.where(y_seq == 0)[0]
    one_indices = np.where(y_seq == 1)[0]

    # Find the number of samples for label 0
    num_zeros = len(zero_indices)

    # Randomly sample an equal number of samples from label 1
    np.random.seed(42)  # Set seed for reproducibility
    sampled_one_indices = np.random.choice(one_indices, num_zeros, replace=False)

    # Combine the indices of label 0 and sampled label 1
    balanced_indices = np.concatenate([zero_indices, sampled_one_indices])

    # Shuffle the balanced indices to avoid any ordering issues
    np.random.shuffle(balanced_indices)

    # Subset X_seq and y_seq based on the balanced indices
    X_seq_balanced = X_seq[balanced_indices]
    y_seq_balanced = y_seq[balanced_indices]

    return X_seq_balanced, y_seq_balanced


In [14]:
X_seq_smart, y_seq_smart = balance_data(X_seq_smart, y_seq_smart)

In [15]:
print(np.unique(y_seq_smart, return_counts=True))

(array([0, 1]), array([1350, 1350]))


In [16]:
X_train_smart, X_test_smart, y_train_smart, y_test_smart = train_test_split(X_smart, y_smart, test_size=0.3, random_state = 42)
X_train_seq_smart, X_test_seq_smart, y_train_seq_smart, y_test_seq_smart = train_test_split(X_seq_smart, y_seq_smart, test_size = 0.3, shuffle = True)

In [17]:
X_train_seq_smart.shape

(1890, 10, 4)

In [18]:
scaler = joblib.load('scaler_m0.sav')

X_train_smart = scaler.fit_transform(X_train_smart)
X_test_smart = scaler.transform(X_test_smart)

In [19]:
train_means = np.mean(X_train_seq_smart, axis=(0, 1))  # Mean of each feature across training samples and timesteps
train_stds = np.std(X_train_seq_smart, axis=(0, 1))    # Standard deviation of each feature across training samples and timesteps

# Handle case where std is zero (to avoid division by zero)
train_stds[train_stds == 0] = 1e-8

# Standardize the training set
X_train_seq_smart = (X_train_seq_smart - train_means) / train_stds

# Standardize the test set using the training set's mean and std
X_test_seq_smart = (X_test_seq_smart - train_means) / train_stds

In [20]:
np.shape(X_train_seq_smart)

(1890, 10, 4)

In [21]:
np.shape(X_test_seq_smart)

(810, 10, 4)

In [22]:
np.shape(X_train_smart)

(103223, 4)

In [23]:
# class_weights = class_weight.compute_class_weight('balanced',
#                                                  classes = np.unique(y_train_smart),
#                                                  y = y_train_smart)
# class_weights = dict(enumerate(class_weights))


# seq_class_weights = class_weight.compute_class_weight('balanced',
#                                                  classes = np.unique(y_train_seq_smart),
#                                                  y = y_train_seq_smart)
# seq_class_weights = dict(enumerate(seq_class_weights))

In [24]:
oversample = SMOTE()
X_train_smote, y_train_smote = oversample.fit_resample(X_train_smart, y_train_smart) 
# X_train_smote, y_train_smote = X_train, y_train_smart

In [25]:
np.unique(y_train_smote, return_counts = True)

(array([0., 1.]), array([70127, 70127]))

In [26]:
##MLP

print("-----MLP-------")

mlp = load_model('mlp_m0.h5')

mlp.compile(optimizer='adam',
                loss=BinaryCrossentropy(from_logits=False),
                metrics=['accuracy'])

es = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)

mlp_hist = mlp.fit(X_train_smote, y_train_smote, epochs=100, callbacks = [es], validation_split=0.2, batch_size = 32)

-----MLP-------
Epoch 1/100


2024-10-15 09:34:11.887017: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 09:34:11.892716: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


3507/3507 [==============================] - 9s 2ms/step - loss: 0.9722 - accuracy: 0.6809 - val_loss: 1.1012 - val_accuracy: 0.3989
Epoch 2/100
3507/3507 [==============================] - 9s 2ms/step - loss: 0.5948 - accuracy: 0.6960 - val_loss: 0.7398 - val_accuracy: 0.4821
Epoch 3/100
3507/3507 [==============================] - 10s 3ms/step - loss: 0.5823 - accuracy: 0.7006 - val_loss: 0.8207 - val_accuracy: 0.4285
Epoch 4/100
3507/3507 [==============================] - 9s 3ms/step - loss: 0.5687 - accuracy: 0.7096 - val_loss: 0.6663 - val_accuracy: 0.5654
Epoch 5/100
3507/3507 [==============================] - 9s 2ms/step - loss: 0.5593 - accuracy: 0.7160 - val_loss: 0.9183 - val_accuracy: 0.4103
Epoch 6/100
3507/3507 [==============================] - 9s 3ms/step - loss: 0.5482 - accuracy: 0.7265 - val_loss: 0.9654 - val_accuracy: 0.4127
Epoch 7/100
3507/3507 [==============================] - 7s 2ms/step - loss: 0.5357 - accuracy: 0.7390 - val_loss: 0.7611 - val_accuracy: 0.4

In [27]:
##MLP
print("-----MLP-------")
threshold = 0.5
mlp_preds = mlp.predict(X_test_smart, batch_size = 32)
mlp_preds = (mlp_preds >= threshold).astype(int)

print("--------Smart--------")
print("ACCURACY: ", accuracy_score(y_test_smart, mlp_preds))
print("CLASSIFICATION REPORT:\n", classification_report(y_test_smart, mlp_preds))


-----MLP-------
 666/1383 [=============>................] - ETA: 0s

1383/1383 [==============================] - 2s 1ms/step
--------Smart--------
ACCURACY:  0.7734804132100636
CLASSIFICATION REPORT:
               precision    recall  f1-score   support

         0.0       0.87      0.79      0.82     29873
         1.0       0.63      0.75      0.68     14366

    accuracy                           0.77     44239
   macro avg       0.75      0.77      0.75     44239
weighted avg       0.79      0.77      0.78     44239



In [28]:
##LSTM

print("-----LSTM-------")

lstm = load_model('lstm_m0.h5')

lstm.compile(
    loss = BinaryCrossentropy(from_logits = False),
    optimizer = Adam(learning_rate = 0.001),
    metrics = ['accuracy'])

es = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
lstm_hist = lstm.fit(X_train_seq_smart, y_train_seq_smart, batch_size = 32, validation_split = 0.2,
        callbacks = [es], epochs = 1000)


-----LSTM-------
Epoch 1/1000
48/48 [==============================] - 2s 31ms/step - loss: 1.8008 - accuracy: 0.5754 - val_loss: 0.6853 - val_accuracy: 0.6587
Epoch 2/1000
48/48 [==============================] - 7s 155ms/step - loss: 0.6655 - accuracy: 0.6607 - val_loss: 0.6100 - val_accuracy: 0.7487
Epoch 3/1000
48/48 [==============================] - 2s 33ms/step - loss: 0.5851 - accuracy: 0.7196 - val_loss: 0.5470 - val_accuracy: 0.7751
Epoch 4/1000
48/48 [==============================] - 1s 25ms/step - loss: 0.5059 - accuracy: 0.7626 - val_loss: 0.4720 - val_accuracy: 0.7989
Epoch 5/1000
48/48 [==============================] - 2s 34ms/step - loss: 0.4385 - accuracy: 0.7937 - val_loss: 0.4284 - val_accuracy: 0.7831
Epoch 6/1000
48/48 [==============================] - 2s 36ms/step - loss: 0.3930 - accuracy: 0.8142 - val_loss: 0.4343 - val_accuracy: 0.8175
Epoch 7/1000
48/48 [==============================] - 1s 23ms/step - loss: 0.3587 - accuracy: 0.8386 - val_loss: 0.3703 - va

In [29]:
print("-----LSTM-------")

lstm_preds = lstm.predict(X_test_seq_smart, batch_size=32)
lstm_preds = (lstm_preds >= threshold).astype(int)

print("--------Smart--------")
print("ACCURACY: ", accuracy_score(y_test_seq_smart, lstm_preds))
print("CLASSIFICATION REPORT:\n", classification_report(y_test_seq_smart, lstm_preds))

-----LSTM-------
26/26 [==============================] - 0s 4ms/step
--------Smart--------
ACCURACY:  0.9160493827160494
CLASSIFICATION REPORT:
               precision    recall  f1-score   support

           0       0.90      0.93      0.91       392
           1       0.93      0.90      0.92       418

    accuracy                           0.92       810
   macro avg       0.92      0.92      0.92       810
weighted avg       0.92      0.92      0.92       810



In [ ]:
dt = joblib.load('dt_m0.pkl')
dt.fit(X_train_smote, y_train_smote)

dt_preds = dt.predict(X_test_smart)

print("--------Smart--------")
print("ACCURACY: ", accuracy_score(y_test_smart, dt_preds))
print("CLASSIFICATION REPORT:\n", classification_report(y_test_smart, dt_preds))


--------Smart--------
ACCURACY:  0.7350527814824024
CLASSIFICATION REPORT:
               precision    recall  f1-score   support

         0.0       0.77      0.87      0.82     29873
         1.0       0.63      0.44      0.52     14366

    accuracy                           0.74     44239
   macro avg       0.70      0.66      0.67     44239
weighted avg       0.72      0.74      0.72     44239

